In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Data principal
maindata=np.loadtxt('Time_Obs_Calc_Resi_PW_PE.dat')
time=maindata[:,0] # Time in JD 
obs=maindata[:,1]  # differencial magnitude observed
calc=maindata[:,2] # differencial magnitude calculated  
resi=maindata[:,3] # residuals
pw=maindata[:,4]   # point weight
pe=maindata[:,5]   # point error

# Spectral window data
swdata=np.loadtxt('SPECTRAL_WINDOW.fou')
freq=swdata[:,0]  # frequency in c/d
amp=swdata[:,1]   # amplitude

#TF original data
tfdata=np.loadtxt('TF_originaldata.fou')
freq_tf=tfdata[:,0]  # frequency in c/d
amp_tf=tfdata[:,1]   # amplitude
#TF residual data
tfresidata=np.loadtxt('TF_res.fou')
freq_tf_res=tfresidata[:,0]  # frequency in c/d
amp_tf_res=tfresidata[:,1]   # amplitude 


#Frecuencias para fitteo, en donde las dos primeras son las frecuencies independientes, las demas son armónicos y combinaciones
freq_fit=np.loadtxt('Frequencies_Amp_Phase.per')
freqs=freq_fit[:,0]  # frequency in c/d
amps=freq_fit[:,1]   # amplitude    
phases=freq_fit[:,2] # phase in radians
#Number of frequencies
n_freq=len(freqs)

#Ruido y SNR de frecuencias, en donde las dos primeras son las frecuencies independientes, las demas son armónicos y combinaciones
fr_ns_snr=np.loadtxt('Freqs_noise_snr.dat')
freqs_ns=fr_ns_snr[:,0]  # frequency in c/d
noise_ns=fr_ns_snr[:,1]  # noise
snr_ns=fr_ns_snr[:,2]    # SNR

#Frecuencias principales
main_freqs=np.array([freqs[0],freqs[1]])
main_freqs_snr=np.array([snr_ns[0],snr_ns[1]])


Main Frequencies (c/d):  [ 5.83462992 19.72660593 40.0088909 ]
Main Frequencies SNR:  [ 40.13958 148.79367 325.31531]


In [10]:
# ============================================================================
# IDENTIFICACIÓN DE FRECUENCIAS FUNDAMENTALES
# ============================================================================
# Las frecuencias fundamentales son aquellas que tienen múltiplos (armónicos)
# en el conjunto de frecuencias proporcionado.
# ============================================================================

import numpy as np

# Datos proporcionados
frecuencias = freqs

# Función para identificar frecuencias fundamentales
def identificar_fundamentales(freqs, tolerancia_porcentual=1.0):
    """
    Identifica frecuencias fundamentales basándose en relaciones armónicas.
    
    Parámetros:
    freqs: lista de frecuencias
    tolerancia_porcentual: tolerancia para considerar múltiplos (en porcentaje)
    
    Retorna:
    lista de frecuencias fundamentales identificadas
    """
    # Ordenar frecuencias de menor a mayor
    freqs_sorted = sorted(frecuencias)
    n = len(freqs_sorted)
    fundamentales = []
    
    # Convertir tolerancia porcentual a relativa
    tolerancia_relativa = tolerancia_porcentual / 100.0
    
    # Marcar si una frecuencia ha sido identificada como armónico
    es_armonico = [False] * n
    
    for i in range(n):
        if es_armonico[i]:
            continue
            
        f_base = freqs_sorted[i]
        # Verificar si f_base es armónico de alguna frecuencia fundamental ya identificada
        es_armonico_de_alguno = False
        
        for f_fund in fundamentales:
            # Verificar si f_base es múltiplo de f_fund
            relacion = f_base / f_fund
            # Buscar un número entero cercano
            entero_cercano = round(relacion)
            if entero_cercano > 1:  # Solo si es múltiplo (no la fundamental misma)
                error = abs(relacion - entero_cercano) / entero_cercano
                if error <= tolerancia_relativa:
                    es_armonico_de_alguno = True
                    break
        
        if not es_armonico_de_alguno:
            # Verificar si tiene armónicos en la lista
            tiene_armonicos = False
            for j in range(i+1, n):
                f_otra = freqs_sorted[j]
                relacion = f_otra / f_base
                entero_cercano = round(relacion)
                if entero_cercano > 1:
                    error = abs(relacion - entero_cercano) / entero_cercano
                    if error <= tolerancia_relativa:
                        tiene_armonicos = True
                        es_armonico[j] = True
            
            if tiene_armonicos:
                fundamentales.append(f_base)
            else:
                # Si no tiene armónicos pero está en un rango bajo, podría ser fundamental
                if f_base < 100:  # Umbral arbitrario para bajas frecuencias
                    fundamentales.append(f_base)
    
    return sorted(fundamentales)

# Identificar frecuencias fundamentales con 1% de tolerancia
fundamentales = identificar_fundamentales(frecuencias, tolerancia_porcentual=1.0)

# Mostrar resultados
print("=" * 60)
print("ANÁLISIS DE FRECUENCIAS FUNDAMENTALES")
print("=" * 60)
print(f"\nTotal de frecuencias: {len(frecuencias)}")
print(f"Frecuencias fundamentales identificadas: {len(fundamentales)}")
print("\n" + "-" * 60)
print("FRECUENCIAS FUNDAMENTALES (Hz):")
print("-" * 60)
for i, f in enumerate(fundamentales, 1):
    print(f"F{i:2d}: {f:.6f} Hz")

# Análisis adicional: agrupar armónicos por fundamental
print("\n" + "-" * 60)
print("POSIBLES GRUPOS DE ARMÓNICOS:")
print("-" * 60)

for f_fund in fundamentales:
    armonicos = []
    for f in frecuencias:
        if abs(f - f_fund) < 0.001:  # Evitar incluir la fundamental misma
            continue
        relacion = f / f_fund
        entero_cercano = round(relacion)
        if entero_cercano > 0:
            error = abs(relacion - entero_cercano) / entero_cercano
            if error <= 0.01:  # 1% de tolerancia
                armonicos.append((f, entero_cercano))
    
    if armonicos:
        print(f"\nFundamental: {f_fund:.6f} Hz")
        print("Armónicos encontrados:")
        for f_arm, orden in sorted(armonicos, key=lambda x: x[1]):
            print(f"  Orden {orden:2d}: {f_arm:.6f} Hz (relación: {f_arm/f_fund:.3f})")

print("\n" + "=" * 60)
print("RESUMEN:")
print("=" * 60)
print(f"1. Se analizaron {len(frecuencias)} frecuencias en total")
print(f"2. Se identificaron {len(fundamentales)} frecuencias fundamentales")
print(f"3. Las fundamentales están en el rango: {min(fundamentales):.2f} - {max(fundamentales):.2f} Hz")
print(f"4. Las {len(frecuencias) - len(fundamentales)} frecuencias restantes son probablemente armónicos")
print("=" * 60)

ANÁLISIS DE FRECUENCIAS FUNDAMENTALES

Total de frecuencias: 68
Frecuencias fundamentales identificadas: 16

------------------------------------------------------------
FRECUENCIAS FUNDAMENTALES (Hz):
------------------------------------------------------------
F 1: 5.834630 Hz
F 2: 16.114692 Hz
F 3: 19.726606 Hz
F 4: 25.005557 Hz
F 5: 31.395866 Hz
F 6: 33.056235 Hz
F 7: 37.508335 Hz
F 8: 40.008891 Hz
F 9: 40.008891 Hz
F10: 51.678151 Hz
F11: 60.280062 Hz
F12: 65.292287 Hz
F13: 68.070682 Hz
F14: 107.801734 Hz
F15: 114.725495 Hz
F16: 184.763281 Hz

------------------------------------------------------------
POSIBLES GRUPOS DE ARMÓNICOS:
------------------------------------------------------------

Fundamental: 5.834630 Hz
Armónicos encontrados:
  Orden  2: 11.669260 Hz (relación: 2.000)
  Orden  5: 29.173150 Hz (relación: 5.000)
  Orden  6: 35.007780 Hz (relación: 6.000)
  Orden  8: 46.677039 Hz (relación: 8.000)
  Orden 15: 87.519449 Hz (relación: 15.000)
  Orden 16: 93.076239 Hz (rel

In [13]:
import numpy as np
from typing import List, Tuple, Dict

def es_armonico(f1: float, f2: float, tolerancia: float = 0.05) -> Tuple[bool, float, int]:
    """
    Determina si f2 es armónico de f1.
    
    Args:
        f1: Frecuencia fundamental
        f2: Frecuencia a verificar
        tolerancia: Tolerancia relativa para la relación armónica
        
    Returns:
        Tuple: (es_armonico, relacion, orden)
    """
    if f1 == 0:
        return False, 0, 0
    
    relacion = f2 / f1
    orden = round(relacion)
    
    if orden < 1:
        # f2 es más baja que f1, podría ser subarmónico
        relacion_inv = f1 / f2
        orden_inv = round(relacion_inv)
        error_relativo = abs(relacion_inv - orden_inv) / orden_inv
        
        if error_relativo <= tolerancia:
            # f1 es armónico de f2 (subarmónico)
            return False, relacion_inv, orden_inv
        else:
            return False, relacion, 0
    
    error_relativo = abs(relacion - orden) / orden
    
    if error_relativo <= tolerancia:
        return True, relacion, orden
    else:
        return False, relacion, 0

def buscar_armonicos(fundamental: float, frecuencias: List[float], 
                     max_orden: int = 100, tolerancia: float = 0.05) -> Dict[int, float]:
    """
    Busca armónicos de una frecuencia fundamental en una lista de frecuencias.
    
    Args:
        fundamental: Frecuencia fundamental
        frecuencias: Lista de frecuencias a analizar
        max_orden: Máximo orden armónico a buscar
        tolerancia: Tolerancia para la relación armónica
        
    Returns:
        Dict: {orden: frecuencia} de armónicos encontrados
    """
    armonicos = {}
    
    for f in frecuencias:
        if f == fundamental:
            continue
            
        es_arm, relacion, orden = es_armonico(fundamental, f, tolerancia)
        
        if es_arm and 1 < orden <= max_orden:
            # Verificar que no sea un armónico de orden muy bajo (ya sería otra fundamental)
            if orden > 2 or fundamental < min(frecuencias) * 0.8:
                armonicos[orden] = f
    
    return armonicos

def analizar_tres_fundamentales(frecuencias_principales: List[float], 
                               todas_frecuencias: List[float],
                               nombres: List[str] = None,
                               max_orden: int = 100,
                               tolerancia: float = 0.05):
    """
    Analiza armónicos para tres frecuencias principales.
    
    Args:
        frecuencias_principales: Lista de 3 frecuencias principales
        todas_frecuencias: Lista con todas las frecuencias del espectro
        nombres: Nombres para las frecuencias principales
        max_orden: Máximo orden armónico a buscar
        tolerancia: Tolerancia para la relación armónica
    """
    if len(frecuencias_principales) != 3:
        raise ValueError("Debe proporcionar exactamente 3 frecuencias principales")
    
    if nombres is None:
        nombres = [f"Principal {i+1}" for i in range(3)]
    
    print("=" * 60)
    print("ANÁLISIS DE TRES FRECUENCIAS PRINCIPALES")
    print("=" * 60)
    print(f"Total de frecuencias en el espectro: {len(todas_frecuencias)}")
    print()
    
    for i, (freq, nombre) in enumerate(zip(frecuencias_principales, nombres), 1):
        print("-" * 50)
        print(f"{nombre}: {freq:.6f} Hz")
        print("-" * 50)
        
        # Buscar armónicos
        armonicos = buscar_armonicos(freq, todas_frecuencias, max_orden, tolerancia)
        
        if armonicos:
            print(f"Armónicos encontrados (de {len(armonicos)} órdenes):")
            for orden in sorted(armonicos.keys()):
                f_arm = armonicos[orden]
                relacion = f_arm / freq
                print(f"  Orden {orden:3d}: {f_arm:10.6f} Hz (relación: {relacion:.3f})")
        else:
            print("No se encontraron armónicos directos")
        
        # Buscar subarmónicos (frecuencias de las que esta podría ser armónico)
        print(f"\nSubarmónicos posibles (frecuencias base para esta):")
        encontrados = False
        for f in todas_frecuencias:
            if f >= freq or f == 0:
                continue
                
            es_arm, relacion, orden = es_armonico(f, freq, tolerancia)
            if es_arm and orden > 1:
                print(f"  Podría ser armónico de {f:.6f} Hz (orden {orden}, relación {relacion:.3f})")
                encontrados = True
        
        if not encontrados:
            print("  No se encontraron subarmónicos claros")
        
        print()
    
    # Análisis de relaciones entre las tres principales
    print("-" * 50)
    print("RELACIONES ENTRE LAS TRES FRECUENCIAS PRINCIPALES")
    print("-" * 50)
    
    f1, f2, f3 = frecuencias_principales
    
    # Relación entre cada par
    relaciones = [
        (nombres[0], nombres[1], f2/f1),
        (nombres[0], nombres[2], f3/f1),
        (nombres[1], nombres[2], f3/f2)
    ]
    
    for nombre1, nombre2, relacion in relaciones:
        cociente_aproximado = f"{relacion:.3f}"
        fraccion = aproximar_fraccion(relacion)
        print(f"{nombre1} / {nombre2} = {cociente_aproximado} ≈ {fraccion}")
    
    print()
    
    # Buscar frecuencias que sean combinaciones lineales
    print("BUSQUEDA DE FRECUENCIAS COMBINADAS (sumas/restas):")
    
    combinaciones_esperadas = [
        f1 + f2,
        f1 + f3,
        f2 + f3,
        abs(f1 - f2),
        abs(f1 - f3),
        abs(f2 - f3),
        f1 + f2 + f3,
        f1 + f2 - f3
    ]
    
    combinaciones_encontradas = []
    
    for comb in combinaciones_esperadas:
        # Buscar la frecuencia más cercana en el espectro
        if comb > 0:
            dif_min = float('inf')
            freq_cercana = None
            
            for f in todas_frecuencias:
                dif = abs(f - comb)
                if dif < dif_min and dif/comb < tolerancia:
                    dif_min = dif
                    freq_cercana = f
            
            if freq_cercana is not None:
                error_pct = abs(freq_cercana - comb) / comb * 100
                combinaciones_encontradas.append((comb, freq_cercana, error_pct))
    
    if combinaciones_encontradas:
        for comb_teorica, comb_real, error in combinaciones_encontradas:
            fuente = ""
            if abs(comb_teorica - (f1 + f2)) < 0.001:
                fuente = f"({nombres[0]} + {nombres[1]})"
            elif abs(comb_teorica - (f1 + f3)) < 0.001:
                fuente = f"({nombres[0]} + {nombres[2]})"
            elif abs(comb_teorica - (f2 + f3)) < 0.001:
                fuente = f"({nombres[1]} + {nombres[2]})"
            elif abs(comb_teorica - abs(f1 - f2)) < 0.001:
                fuente = f"|{nombres[0]} - {nombres[1]}|"
            
            print(f"  {fuente}: Teórica = {comb_teorica:.3f} Hz, Encontrada = {comb_real:.3f} Hz (error: {error:.1f}%)")
    else:
        print("  No se encontraron combinaciones lineales claras")
    
    print()
    
    # Resumen de cobertura armónica
    print("=" * 60)
    print("RESUMEN DE COBERTURA ARMÓNICA")
    print("=" * 60)
    
    todas_freq_set = set(todas_frecuencias)
    explicadas = set()
    
    for i, freq in enumerate(frecuencias_principales):
        armonicos = buscar_armonicos(freq, todas_frecuencias, max_orden, tolerancia)
        explicados = [armonicos[orden] for orden in armonicos]
        explicadas.update(explicados)
        
        print(f"{nombres[i]} ({freq:.3f} Hz): {len(armonicos)} armónicos, "
              f"explica {len(explicados)} frecuencias del espectro")
    
    print(f"\nTotal de frecuencias explicadas por las 3 principales: {len(explicadas)}/{len(todas_frecuencias)}")
    
    if len(explicadas) < len(todas_frecuencias):
        no_explicadas = [f for f in todas_frecuencias if f not in explicadas]
        print(f"Frecuencias no explicadas ({len(no_explicadas)}):")
        for f in sorted(no_explicadas):
            print(f"  {f:.3f} Hz")

def aproximar_fraccion(valor: float, max_denominador: int = 100) -> str:
    """
    Aproxima un número decimal a una fracción simple.
    """
    from fractions import Fraction
    frac = Fraction(valor).limit_denominator(max_denominador)
    return f"{frac.numerator}/{frac.denominator}"

def analizar_modulacion_orbital(f_orbital: float, f_portadoras: List[float],
                               todas_frecuencias: List[float], 
                               orden_modulacion: int = 3,
                               tolerancia: float = 0.05):
    """
    Analiza frecuencias generadas por modulación orbital.
    """
    print("\n" + "=" * 60)
    print("ANÁLISIS DE MODULACIÓN ORBITAL")
    print("=" * 60)
    print(f"Frecuencia orbital (moduladora): {f_orbital:.6f} Hz")
    print()
    
    for i, f_portadora in enumerate(f_portadoras, 1):
        print(f"Portadora {i}: {f_portadora:.6f} Hz")
        print(f"Bandas laterales esperadas (± n × {f_orbital:.3f} Hz):")
        
        bandas_encontradas = []
        
        for n in range(1, orden_modulacion + 1):
            # Bandas laterales superiores e inferiores
            banda_sup = f_portadora + n * f_orbital
            banda_inf = f_portadora - n * f_orbital
            
            # Buscar en las frecuencias reales
            for banda_teorica in [banda_sup, banda_inf]:
                if banda_teorica <= 0:
                    continue
                    
                # Buscar la frecuencia más cercana
                for f_real in todas_frecuencias:
                    error_rel = abs(f_real - banda_teorica) / banda_teorica
                    if error_rel < tolerancia:
                        bandas_encontradas.append((n, banda_teorica, f_real, error_rel*100))
                        break
        
        # Ordenar y mostrar resultados
        bandas_encontradas.sort(key=lambda x: x[0])
        
        if bandas_encontradas:
            for n, teorica, real, error in bandas_encontradas:
                signo = "+" if teorica > f_portadora else "-"
                print(f"  Orden {n} ({signo}): Teórica = {teorica:.3f} Hz, "
                      f"Encontrada = {real:.3f} Hz (error: {error:.1f}%)")
        else:
            print("  No se encontraron bandas laterales claras")
        print()

# ============================================================================
# DATOS DE EJEMPLO CON TUS FRECUENCIAS
# ============================================================================

# Todas las frecuencias de tu análisis original
todas_las_frecuencias = [
    5.834630, 16.114692, 19.726606, 25.005557, 31.395866, 33.056235,
    37.508335, 40.008891, 40.008891, 51.678151, 60.280062, 65.292287,
    68.070682, 107.801734, 114.725495, 184.763281,
    # Frecuencias adicionales de tus listas de armónicos (agregadas manualmente)
    11.669260, 29.173150, 35.007780, 46.677039, 87.519449, 93.076239,
    99.724383, 128.917537, 139.753279, 158.924205, 179.206490, 199.488775,
    209.490998, 239.219827, 279.228718, 287.286064, 298.955324, 319.237609,
    339.519894, 359.246499, 379.250945, 399.255390, 418.959769, 439.168704,
    458.713048, 478.717493, 498.336297, 519.170927, 538.730829, 558.457435,
    579.183152, 598.336297, 618.915315, 638.896421, 658.757502, 678.484108,
    698.210714, 718.215159, 738.219605, 758.224050, 778.228496, 837.686153,
    918.259614, 79.890865, 119.723272, 135.585686, 155.034452, 165.870194,
    219.493221, 259.502112, 259.445432
]

# Eliminar duplicados y ordenar
todas_las_frecuencias = sorted(list(set(todas_las_frecuencias)))

# ============================================================================
# HIPÓTESIS 1: Las 3 frecuencias principales identificadas
# ============================================================================

print("ANÁLISIS BASADO EN HIPÓTESIS DE SISTEMA BINARIO")
print("=" * 60)

# Opción 1: Las 3 que identificamos como más probables
frecuencias_principales_1 = [5.834630, 40.008891, 19.726606]  # f_orbital, f_A, f_B
nombres_1 = ["F_orbital (5.83 Hz)", "Componente A (40.01 Hz)", "Componente B (19.73 Hz)"]

analizar_tres_fundamentales(frecuencias_principales_1, todas_las_frecuencias, 
                           nombres=nombres_1, max_orden=100, tolerancia=0.05)

# Análisis específico de modulación orbital
analizar_modulacion_orbital(5.834630, [40.008891, 19.726606, 31.395866], 
                           todas_las_frecuencias, orden_modulacion=5, tolerancia=0.05)

# ============================================================================
# HIPÓTESIS 2: Otra combinación posible
# ============================================================================

print("\n" + "=" * 60)
print("ANÁLISIS ALTERNATIVO: Otra combinación de 3 principales")
print("=" * 60)

# Opción 2: Otra combinación interesante
frecuencias_principales_2 = [5.834630, 33.056235, 65.292287]
nombres_2 = ["Base 1 (5.83 Hz)", "Base 2 (33.06 Hz)", "Base 3 (65.29 Hz)"]

analizar_tres_fundamentales(frecuencias_principales_2, todas_las_frecuencias,
                           nombres=nombres_2, max_orden=50, tolerancia=0.05)

# ============================================================================
# ANÁLISIS DE ARMÓNICOS COMPLETO PARA LAS 3 PRINCIPALES (HIPÓTESIS 1)
# ============================================================================

print("\n" + "=" * 60)
print("ANÁLISIS DETALLADO POR ORDEN ARMÓNICO")
print("=" * 60)

for freq, nombre in zip(frecuencias_principales_1, nombres_1):
    print(f"\n{nombre}: {freq:.6f} Hz")
    print("-" * 40)
    
    armonicos = buscar_armonicos(freq, todas_las_frecuencias, max_orden=200, tolerancia=0.03)
    
    # Agrupar por rangos de orden
    rangos = {
        "Bajos (2-10)": [],
        "Medios (11-50)": [],
        "Altos (51-100)": [],
        "Muy altos (101+)": []
    }
    
    for orden, f_arm in armonicos.items():
        if 2 <= orden <= 10:
            rangos["Bajos (2-10)"].append((orden, f_arm))
        elif 11 <= orden <= 50:
            rangos["Medios (11-50)"].append((orden, f_arm))
        elif 51 <= orden <= 100:
            rangos["Altos (51-100)"].append((orden, f_arm))
        else:
            rangos["Muy altos (101+)"].append((orden, f_arm))
    
    for rango_nombre, lista_armonicos in rangos.items():
        if lista_armonicos:
            print(f"{rango_nombre}: {len(lista_armonicos)} armónicos")
            # Mostrar algunos ejemplos
            lista_armonicos.sort()
            for orden, f_arm in lista_armonicos[:5]:  # Mostrar primeros 5
                relacion = f_arm / freq
                print(f"  Orden {orden}: {f_arm:.3f} Hz (x{relacion:.3f})")
            if len(lista_armonicos) > 5:
                print(f"  ... y {len(lista_armonicos)-5} más")

# ============================================================================
# TABLA RESUMEN COMPARATIVA
# ============================================================================

print("\n" + "=" * 60)
print("TABLA RESUMEN COMPARATIVA")
print("=" * 60)
print(f"{'Frecuencia':<20} {'Valor (Hz)':<15} {'Armónicos':<12} {'Cobertura':<15}")
print("-" * 60)

for freq, nombre in zip(frecuencias_principales_1, nombres_1):
    armonicos = buscar_armonicos(freq, todas_las_frecuencias, max_orden=200, tolerancia=0.03)
    num_armonicos = len(armonicos)
    
    # Calcular cuántas frecuencias del total son explicadas por esta fundamental
    explicadas = set()
    for orden, f_arm in armonicos.items():
        # Encontrar la frecuencia más cercana en la lista original
        for f in todas_las_frecuencias:
            if abs(f - f_arm)/f_arm < 0.05:
                explicadas.add(f)
    
    cobertura = f"{len(explicadas)}/{len(todas_las_frecuencias)}"
    
    print(f"{nombre:<20} {freq:<15.3f} {num_armonicos:<12} {cobertura:<15}")

print("\nNOTA: La 'cobertura' muestra cuántas frecuencias del espectro total")
print("pueden explicarse como armónicos de cada frecuencia principal.")

ANÁLISIS BASADO EN HIPÓTESIS DE SISTEMA BINARIO
ANÁLISIS DE TRES FRECUENCIAS PRINCIPALES
Total de frecuencias en el espectro: 66

--------------------------------------------------
F_orbital (5.83 Hz): 5.834630 Hz
--------------------------------------------------
Armónicos encontrados (de 44 órdenes):
  Orden   5:  29.173150 Hz (relación: 5.000)
  Orden   6:  35.007780 Hz (relación: 6.000)
  Orden   7:  40.008891 Hz (relación: 6.857)
  Orden   8:  46.677039 Hz (relación: 8.000)
  Orden   9:  51.678151 Hz (relación: 8.857)
  Orden  10:  60.280062 Hz (relación: 10.331)
  Orden  11:  65.292287 Hz (relación: 11.190)
  Orden  12:  68.070682 Hz (relación: 11.667)
  Orden  14:  79.890865 Hz (relación: 13.693)
  Orden  15:  87.519449 Hz (relación: 15.000)
  Orden  16:  93.076239 Hz (relación: 15.952)
  Orden  17:  99.724383 Hz (relación: 17.092)
  Orden  18: 107.801734 Hz (relación: 18.476)
  Orden  20: 114.725495 Hz (relación: 19.663)
  Orden  21: 119.723272 Hz (relación: 20.519)
  Orden  22